In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.0/38.8 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.1/38.8 MB 880.9 kB/s eta 0:00:44
   ---------------------------------------- 0.1/38.8 MB 880.9 kB/s eta 0:00:44
   ---------------------------------------- 0.1/38.8 MB 880.9 kB/s eta 0:00:44
   ---------------------------------------- 0.1/38.8 MB 481.4 kB/s eta 0:01:21
   ---------------------------------------- 0.1/38.8 MB 448.2 kB/s eta 0:01:27
   ---------------------------------------- 0.1/38.8 MB 448.2 kB/s eta 0:01:27
   ---------------------------------------- 0.1/38.8 MB 448.2 kB/s eta 0:01:27
 

In [152]:
import cv2

In [5]:
# Check OpenCV version
print("OpenCV version:", cv2.__version__)

OpenCV version: 4.10.0


In [7]:
import numpy as np
import pandas as pd

In [8]:
# Load the image
image = cv2.imread('road_image.jpg')

# Iteration 1

In [10]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [12]:
# Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [13]:
# Perform edge detection using Canny
edges = cv2.Canny(blurred, 50, 150)

In [14]:
# Detect lines using Hough Line Transform
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

In [15]:
# Draw the detected lines on the original image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

In [18]:
# Show the image with detected lines
cv2.imshow("Detected Lines", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# Calculate the width of the road (in pixels)
# Here you would need to find the two main road lines and calculate the distance between them.
# This part of the code is highly dependent on the structure of the lines detected.
# Assuming the first two lines are the road edges, you can compute the Euclidean distance:
if len(lines) >= 2:
    line1 = lines[0][0]
    line2 = lines[1][0]

    # Calculate the distance between the two lines
    def distance_between_lines(line1, line2):
        # Calculate the midpoint of each line
        midpoint1 = ((line1[0] + line1[2]) / 2, (line1[1] + line1[3]) / 2)
        midpoint2 = ((line2[0] + line2[2]) / 2, (line2[1] + line2[3]) / 2)
        
        # Use Euclidean distance formula
        distance = np.sqrt((midpoint1[0] - midpoint2[0])**2 + (midpoint1[1] - midpoint2[1])**2)
        return distance

    road_width = distance_between_lines(line1, line2)
    print(f'Road width in pixels: {road_width}')

Road width in pixels: 730.8778967789353


# Iteration 2

In [20]:
# Resize the image for faster processing (optional)
scale_percent = 50  # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

In [21]:
# Convert to grayscale
gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

In [22]:
# Apply Gaussian blur to smooth the image
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [23]:
# Apply Canny Edge Detection
edges = cv2.Canny(blurred, 50, 150)

In [24]:
# Define a region of interest (ROI) to focus on the road area
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [25]:
# Define the vertices for the region of interest (this will vary depending on your image)
height = resized_image.shape[0]
width = resized_image.shape[1]
roi_vertices = [(0, height), (width // 2, height // 2), (width, height)]

In [26]:
# Apply the ROI mask
cropped_edges = region_of_interest(edges, np.array([roi_vertices], np.int32))

In [27]:
# Perform Hough Line Transform to detect lines
lines = cv2.HoughLinesP(cropped_edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=50)

In [28]:
# Draw the detected lines on a copy of the original image
line_image = np.copy(resized_image)

In [29]:
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 3)

In [30]:
# Display the lines on the image
cv2.imshow("Road with Detected Lines", line_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Iteration 3

In [31]:
# Convert to grayscale (if necessary)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [32]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [33]:
# Apply a threshold to segment the road (you might need to adjust the threshold values)
_, road_segment = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY_INV)

In [34]:
# Use morphological operations to clean up the noise (if needed)
kernel = np.ones((5, 5), np.uint8)
road_segment = cv2.morphologyEx(road_segment, cv2.MORPH_CLOSE, kernel)

In [35]:
# Find the contours of the segmented road area
contours, _ = cv2.findContours(road_segment, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [36]:
# Filter and find the largest contour, assuming it's the road
largest_contour = max(contours, key=cv2.contourArea)

In [37]:
# Draw the contour on the original image (for visualization)
contour_image = image.copy()
cv2.drawContours(contour_image, [largest_contour], -1, (0, 255, 0), 3)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

In [38]:
# Get the extreme left and right points on the road's contour
leftmost = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
rightmost = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])

In [39]:
# Draw these points on the image (optional, for visualization)
cv2.circle(contour_image, leftmost, 10, (255, 0, 0), -1)
cv2.circle(contour_image, rightmost, 10, (255, 0, 0), -1)

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [40]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [41]:
# Display the image with the detected road edges
cv2.imshow("Road with Contours", contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
# Print the road width in pixels
print(f"Road width in pixels: {road_width_pixels}")

Road width in pixels: 1467.2429928270233


In [43]:
# Conversion from pixels to meters (if you know the scale, e.g., pixels per meter)
# Example: If 1 pixel = 0.02 meters, then:
pixels_per_meter = 0.02  # You need to determine this based on reference object or camera info
road_width_meters = road_width_pixels * pixels_per_meter
print(f"Estimated Road width in meters: {road_width_meters}")

Estimated Road width in meters: 29.344859856540467


# Iteration 4

In [44]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [45]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [46]:
# Apply thresholding to segment the road (you might need to adjust the threshold value)
_, road_segment = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)

In [47]:
# Use morphological operations to clean up the noise (if needed)
kernel = np.ones((5, 5), np.uint8)
road_segment = cv2.morphologyEx(road_segment, cv2.MORPH_CLOSE, kernel)

In [48]:
# Find the contours of the segmented road area
contours, _ = cv2.findContours(road_segment, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [49]:
# Filter and find the largest contour, assuming it's the road
largest_contour = max(contours, key=cv2.contourArea)

In [50]:
# Get the extreme left and right points on the road's contour
leftmost = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
rightmost = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])

In [51]:
# Draw these points on the image (optional, for visualization)
contour_image = image.copy()
cv2.circle(contour_image, leftmost, 10, (255, 0, 0), -1)
cv2.circle(contour_image, rightmost, 10, (255, 0, 0), -1)

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [52]:
# Draw the detected left and right road boundaries
cv2.line(contour_image, leftmost, rightmost, (0, 255, 0), 3)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

In [53]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [54]:
# Display the image with the detected road edges
cv2.imshow("Road with Detected Edges", contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [55]:
# Print the road width in pixels
print(f"Road width in pixels: {road_width_pixels}")

Road width in pixels: 1467.2429928270233


In [56]:
# Step 2: Conversion from Pixels to Meters using the Car's Width
# Assuming the car in the image is about 1.7 meters wide (standard sedan width)
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (adjust this based on the image)

In [57]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [59]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter
print(f"Road width in meters: {road_width_meters:.4f} meters")

Road width in meters: 16.6288 meters


# Iteration 5

In [61]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [62]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [63]:
# Apply thresholding to segment the road (you might need to adjust the threshold value)
_, road_segment = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)

In [64]:
# Use morphological operations to clean up the noise (if needed)
kernel = np.ones((5, 5), np.uint8)
road_segment = cv2.morphologyEx(road_segment, cv2.MORPH_CLOSE, kernel)

In [65]:
# Find the contours of the segmented road area
contours, _ = cv2.findContours(road_segment, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [66]:
# Filter and find the largest contour, assuming it's the road
largest_contour = max(contours, key=cv2.contourArea)

In [67]:
# Get the extreme left and right points on the road's contour
leftmost = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
rightmost = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])

In [68]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [69]:
# Display the image with the detected road edges
contour_image = image.copy()

In [70]:
# Draw the detected left and right road boundaries
cv2.circle(contour_image, leftmost, 10, (0, 255, 0), -1)  # Leftmost point
cv2.circle(contour_image, rightmost, 10, (0, 255, 0), -1)  # Rightmost point

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

In [71]:
# Draw a line between the leftmost and rightmost points
cv2.line(contour_image, leftmost, rightmost, (255, 0, 0), 2)

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [  0, 255,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [72]:
# Step 2: Conversion from Pixels to Meters using the Car's Width
# Assuming the car in the image is about 1.7 meters wide (standard sedan width)
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (adjust this based on the image)

In [73]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [74]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter
print(f"Road width in pixels: {road_width_pixels}")
print(f"Road width in meters: {road_width_meters:.2f} meters")

Road width in pixels: 1467.2429928270233
Road width in meters: 16.63 meters


In [75]:
# Add text annotation showing the road width in meters
font = cv2.FONT_HERSHEY_SIMPLEX
text = f"Road Width: {road_width_meters:.2f} m"
cv2.putText(contour_image, text, (leftmost[0], leftmost[1] - 10), font, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[  0, 255,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [  0, 255,   0]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [77]:
# Display the final image with annotations
cv2.imshow("Road with Detected Edges", contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Iteration 6

In [78]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [79]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [80]:
# Use color thresholding to detect white lines
# Define lower and upper bounds for white color in BGR format
lower_white = np.array([200, 200, 200])
upper_white = np.array([255, 255, 255])

In [81]:
# Create a mask for white regions in the image
mask = cv2.inRange(image, lower_white, upper_white)

In [82]:
# Find contours of the white lines
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [83]:
# Filter and find the largest contours that could represent the road boundaries
road_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]

In [84]:
# Initialize variables to hold the leftmost and rightmost points
leftmost = None
rightmost = None

In [85]:
# Iterate through contours to find the extreme points
for contour in road_contours:
    # Get extreme points
    left_point = tuple(contour[contour[:, :, 0].argmin()][0])
    right_point = tuple(contour[contour[:, :, 0].argmax()][0])
    
    # Update leftmost and rightmost points
    if leftmost is None or left_point[0] < leftmost[0]:
        leftmost = left_point
    if rightmost is None or right_point[0] > rightmost[0]:
        rightmost = right_point

In [86]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [87]:
# Conversion from Pixels to Meters using the car's width
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (you might need to adjust this based on the image)

In [88]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [89]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter

In [90]:
# Draw the detected leftmost and rightmost road boundaries
road_image = image.copy()
cv2.circle(road_image, leftmost, 10, (0, 255, 0), -1)  # Leftmost point
cv2.circle(road_image, rightmost, 10, (0, 255, 0), -1)  # Rightmost point

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [91]:
# Draw a line between the leftmost and rightmost points
cv2.line(road_image, leftmost, rightmost, (255, 0, 0), 2)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [92]:
# Add text annotation showing the road width in meters
font = cv2.FONT_HERSHEY_SIMPLEX
text = f"Road Width: {road_width_meters:.2f} m"
cv2.putText(road_image, text, (leftmost[0], leftmost[1] - 10), font, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [95]:
# Display the final image with annotations
cv2.imshow("Road with Detected Edges", road_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [94]:
# Print the calculated width for reference
print(f"Road width in pixels: {road_width_pixels:.2f} pixels")
print(f"Road width in meters: {road_width_meters:.2f} meters")

Road width in pixels: 1300.77 pixels
Road width in meters: 14.74 meters


# Iteration 7

In [96]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [97]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [98]:
# Use color thresholding to detect white lines
lower_white = np.array([200, 200, 200])
upper_white = np.array([255, 255, 255])
mask = cv2.inRange(image, lower_white, upper_white)

In [99]:
# Find contours of the white lines
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [100]:
# Filter and find the largest contours that could represent the road boundaries
road_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]

In [101]:
# Initialize lists to hold leftmost and rightmost points for each section of the road
left_points = []
right_points = []

In [102]:
# Iterate through the contours and collect points for curve fitting
for contour in road_contours:
    # Get extreme points for each contour
    left_point = tuple(contour[contour[:, :, 0].argmin()][0])
    right_point = tuple(contour[contour[:, :, 0].argmax()][0])
    
    # Add points to the lists for curve fitting
    left_points.append(left_point)
    right_points.append(right_point)

In [103]:
# Fit polynomial curves (degree 2) to the left and right points to get the road's curvature
if len(left_points) > 1 and len(right_points) > 1:
    left_curve = np.polyfit([pt[1] for pt in left_points], [pt[0] for pt in left_points], 2)
    right_curve = np.polyfit([pt[1] for pt in right_points], [pt[0] for pt in right_points], 2)
else:
    raise ValueError("Not enough points detected for curve fitting.")

In [104]:
# Create the fitted lines by evaluating the polynomial at different y-values
y_values = np.linspace(0, height, num=height)
left_fitted_points = np.polyval(left_curve, y_values)
right_fitted_points = np.polyval(right_curve, y_values)

In [105]:
# Draw the fitted curves on the image (blue lines)
road_image = image.copy()
for i in range(len(y_values) - 1):
    # Draw left curve
    cv2.line(road_image, (int(left_fitted_points[i]), int(y_values[i])), 
             (int(left_fitted_points[i + 1]), int(y_values[i + 1])), (255, 0, 0), 2)
    # Draw right curve
    cv2.line(road_image, (int(right_fitted_points[i]), int(y_values[i])), 
             (int(right_fitted_points[i + 1]), int(y_values[i + 1])), (255, 0, 0), 2)

In [106]:
# Calculate the road width at a specific point (e.g., in the middle of the image)
mid_y = int(height / 2)
left_x_mid = np.polyval(left_curve, mid_y)
right_x_mid = np.polyval(right_curve, mid_y)

In [107]:
# Calculate road width in pixels
road_width_pixels = abs(right_x_mid - left_x_mid)

In [108]:
# Conversion from Pixels to Meters
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (you might need to adjust this based on the image)

In [109]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [110]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter

In [111]:
# Draw a horizontal line at the middle to represent the road width
cv2.line(road_image, (int(left_x_mid), mid_y), (int(right_x_mid), mid_y), (0, 255, 255), 2)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [112]:
# Add text annotation showing the road width in meters
font = cv2.FONT_HERSHEY_SIMPLEX
text = f"Road Width: {road_width_meters:.2f} m"
cv2.putText(road_image, text, (int(left_x_mid), mid_y - 10), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [113]:
# Display the final image with annotations
cv2.imshow("Road with Fitted Curves and Width", road_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [114]:
# Print the calculated width for reference
print(f"Road width in pixels: {road_width_pixels:.2f} pixels")
print(f"Road width in meters: {road_width_meters:.2f} meters")

Road width in pixels: 14.08 pixels
Road width in meters: 0.16 meters


# Iteration 8

In [115]:
# Convert to HSV to detect black and yellow colors of the boundary
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [116]:
# Define the lower and upper bounds for yellow and black colors
lower_yellow = np.array([15, 100, 100])
upper_yellow = np.array([30, 255, 255])
lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 50])

In [117]:
# Create masks for black and yellow colors
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
mask_black = cv2.inRange(hsv, lower_black, upper_black)

In [118]:
# Combine the masks to get the black-yellow boundary
mask_boundary = cv2.bitwise_or(mask_yellow, mask_black)

In [119]:
# Apply the mask to the image to get the road boundaries
masked_image = cv2.bitwise_and(image, image, mask=mask_boundary)

In [120]:
# Find the contours of the road boundaries
contours, _ = cv2.findContours(mask_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [121]:
# Filter contours by area to remove small noise and focus on large boundaries
boundary_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

In [122]:
# Initialize variables to hold the leftmost and rightmost points for the road
leftmost = None
rightmost = None

In [123]:
# Iterate through contours and find the extreme leftmost and rightmost points
for contour in boundary_contours:
    # Get extreme points for each contour
    left_point = tuple(contour[contour[:, :, 0].argmin()][0])
    right_point = tuple(contour[contour[:, :, 0].argmax()][0])
    
    # Update the leftmost and rightmost points for the road boundary
    if leftmost is None or left_point[0] < leftmost[0]:
        leftmost = left_point
    if rightmost is None or right_point[0] > rightmost[0]:
        rightmost = right_point

In [124]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [125]:
# Conversion from Pixels to Meters using the car's width
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (adjust as necessary)

In [126]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [127]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter

In [128]:
# Draw the detected leftmost and rightmost road boundaries
road_image = image.copy()
cv2.circle(road_image, leftmost, 10, (0, 255, 0), -1)  # Leftmost point
cv2.circle(road_image, rightmost, 10, (0, 255, 0), -1)  # Rightmost point

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [129]:
# Draw a line between the leftmost and rightmost points (representing road width)
cv2.line(road_image, leftmost, rightmost, (255, 0, 0), 2)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [130]:
# Add text annotation showing the road width in meters
font = cv2.FONT_HERSHEY_SIMPLEX
text = f"Road Width: {road_width_meters:.2f} m"
cv2.putText(road_image, text, (int((leftmost[0] + rightmost[0]) / 2), leftmost[1] - 10), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [131]:
# Display the final image with the correct annotation
cv2.imshow("Road Width", road_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [132]:
# Print the calculated width for reference
print(f"Road width in pixels: {road_width_pixels:.2f} pixels")
print(f"Road width in meters: {road_width_meters:.2f} meters")

Road width in pixels: 1260.38 pixels
Road width in meters: 14.28 meters


# Final Iteration

In [133]:
# Convert to HSV to detect black and yellow colors of the boundary
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [134]:
# Define the lower and upper bounds for yellow and black colors
lower_yellow = np.array([15, 100, 100])
upper_yellow = np.array([30, 255, 255])
lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 50])

In [135]:
# Create masks for black and yellow colors
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
mask_black = cv2.inRange(hsv, lower_black, upper_black)

In [136]:
# Combine the masks to get the black-yellow boundary
mask_boundary = cv2.bitwise_or(mask_yellow, mask_black)

In [137]:
# Apply the mask to the image to get the road boundaries
masked_image = cv2.bitwise_and(image, image, mask=mask_boundary)

In [138]:
# Find the contours of the road boundaries
contours, _ = cv2.findContours(mask_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [139]:
# Filter contours by area to remove small noise and focus on large boundaries
boundary_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

In [140]:
# Initialize variables to hold the leftmost and rightmost points for the road
leftmost = None
rightmost = None

In [141]:
# Iterate through contours and find the extreme leftmost and rightmost points
for contour in boundary_contours:
    # Get extreme points for each contour
    left_point = tuple(contour[contour[:, :, 0].argmin()][0])
    right_point = tuple(contour[contour[:, :, 0].argmax()][0])
    
    # Update the leftmost and rightmost points for the road boundary
    if leftmost is None or left_point[0] < leftmost[0]:
        leftmost = left_point
    if rightmost is None or right_point[0] > rightmost[0]:
        rightmost = right_point

In [142]:
# Calculate the road width in pixels (distance between the leftmost and rightmost points)
road_width_pixels = np.linalg.norm(np.array(leftmost) - np.array(rightmost))

In [143]:
# Conversion from Pixels to Meters using the car's width
car_width_meters = 1.7  # Estimated width of the car in meters
car_width_pixels = 150  # Approximate car width in pixels (adjust as necessary)

In [144]:
# Calculate the pixel-to-meter ratio
pixels_per_meter = car_width_pixels / car_width_meters

In [145]:
# Calculate the road width in meters
road_width_meters = road_width_pixels / pixels_per_meter

In [146]:
# Convert road width to feet
road_width_feet = road_width_meters * 3.28084

In [147]:
# Draw the detected leftmost and rightmost road boundaries
road_image = image.copy()
cv2.circle(road_image, leftmost, 10, (0, 255, 0), -1)  # Leftmost point
cv2.circle(road_image, rightmost, 10, (0, 255, 0), -1)  # Rightmost point

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [148]:
# Draw a line between the leftmost and rightmost points (representing road width)
cv2.line(road_image, leftmost, rightmost, (255, 0, 0), 2)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [149]:
# Add text annotation showing the road width in feet
font = cv2.FONT_HERSHEY_SIMPLEX
text = f"Road Width: {road_width_feet:.2f} ft"
cv2.putText(road_image, text, (int((leftmost[0] + rightmost[0]) / 2), leftmost[1] - 10), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

array([[[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 155, 109],
        [205, 155, 109],
        [205, 155, 109],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       [[205, 156, 108],
        [205, 156, 108],
        [205, 156, 108],
        ...,
        [203, 151, 110],
        [203, 151, 110],
        [203, 151, 110]],

       ...,

       [[246, 250, 251],
        [244, 248, 249],
        [238, 242, 243],
        ...,
        [102, 104, 115],
        [103, 105, 116],
        [103, 105, 116]],

       [[239, 243, 244],
        [239, 243, 244],
        [241, 245, 246],
        ...,
        [102, 104, 115],
        [104, 104, 118],
        [104, 104, 118]],

       [[234, 238, 239],
        [238, 242, 243],
        [245, 249, 250],
        ...,
        [103, 105, 116],
        [104, 104, 118],
        [104, 104, 118]]

In [153]:
# Display the final image with the correct annotation
cv2.imshow("Road Width in Feet", road_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [151]:
# Print the calculated width for reference
print(f"Road width in pixels: {road_width_pixels:.2f} pixels")
print(f"Road width in meters: {road_width_meters:.2f} meters")
print(f"Road width in feet: {road_width_feet:.2f} feet")

Road width in pixels: 1260.38 pixels
Road width in meters: 14.28 meters
Road width in feet: 46.86 feet
